In [ ]:
import math
import random

from gym_trading_env.downloader import download
import pandas_ta as ta

In [ ]:
# Download BTC/USDT historical data from Binance and stores it to directory ./data/binance-BTCUSDT-1h.pkl
# download(exchange_names = ["binance"],
#     symbols= ["BTC/USDT"],
#     timeframe= "1d",
#     dir = "data",
#     since= datetime.datetime(year= 2019, month= 12, day=31),
#     until= datetime.datetime(year= 2024, month= 1, day=1)
# )
# Import your fresh data
# df = pd.read_pickle("./data/binance-BTCUSDT-1d.pkl")
# df["close"].plot(figsize=(15, 5))

# MyStrategy = ta.Strategy(
#     name="OHLC4",
#     ta=[
#         {"kind": "ohlc4"},
#     ]
# )

# CustomStrategy = ta.Strategy(
#     name="Momo and Volatility",
#     description="SMA 50,200, BBANDS, RSI, MACD and Volume SMA 20",
#     ta=[
#         {"kind": "sma", "length": 50},
#         {"kind": "sma", "length": 200},
#         {"kind": "bbands", "length": 20},
#         {"kind": "rsi"},
#         {"kind": "macd", "fast": 8, "slow": 21},
#         {"kind": "sma", "close": "volume", "length": 20, "prefix": "VOLUME"},
#     ]
# )

# df.ta.strategy(MyStrategy)
# df.dropna(inplace=True)